<div style="width:100%; background-color: #000041"><a target="_blank" href="http://university.yugabyte.com"><img src="assets/YBU_Logo.png" /></a></div><br>

> **Migrate and Modernize with YugabyteDB Voyager**
>
> Enroll for free at [Yugabyte University](https://university.yugabyte.com/courses/YugabyteDB-Voyager).
>

<br>
This notebook file is:

`05_Migration_Workflow_Migrate_Data.ipynb`


## 🛠️ Requirements
Here are the requirements for this notebook:
- ✅ Create the notebook variables in `01_Lab_Setup.ipynb`, which you previously did
- ✅ Create the `ds_ybu` database, which you previously did
- ☑️ A Python 3.12 or higher kernel for the notebook, *which you need to select right now!!!*
- ☑️ Import the notebook variables, *which you must do next*
- ☑️ Migrate data workflow, *which you must do next*


### Select your notebook kernel
- In the Notebook toolbar, click **Select Kernel**.
<br>
<img width=50% src="assets/01_01_Select_Kernel_Toolbar.png" />

- Next, in the dropdown, select **Python 3.12** or higher.
<br>
<img width=50% src="assets/01_02_Select_Kernel_Dropdown.png" />

That's it!

## ⛑️ Getting help
The best way to get help us from the Yugabyte University team. You can post your question in YugabyteDB Community Slack in the #training or #yb-university channels. Join our [community](https://communityinviter.com/apps/yugabyte-db/register).

## 👣 Set up steps for this notebook
Here are the steps to setup this lab:

- Import the notebook variables

### 👇 Import the notebook variables

> 👉 IMPORTANT! 👈
> 
> Do **NOT** skip running the following cell. 
> 

The following Python cell reads the stored variables created in the `01_Lab_Setup.ipynb` notebook. To run the script, select Execute Cell (Play Arrow) in the left gutter of the cell. 

👇 👇 👇 

In [ ]:
# Use %store -r to read 01_Lab_Setup variables
%store -r MY_YB_PATH
%store -r MY_YB_PATH_DATA

%store -r MY_GITPOD_WORKSPACE_URL

%store -r MY_DB_NAME
%store -r MY_DB_PORT
%store -r MY_MYSQL_DB_NAME

%store -r MY_HOST_IPv4_01
%store -r MY_HOST_IPv4_02
%store -r MY_HOST_IPv4_03

%store -r MY_TSERVER_WEBSERVER_PORT
%store -r MY_MYSQL_PORT

%store -r MY_YB_MASTER_HOST_GITPOD_URL
%store -r MY_YB_TSERVER_HOST_GITPOD_URL

%store -r MY_NOTEBOOK_DIR
%store -r MY_NOTEBOOK_DATA_FOLDER
%store -r MY_NOTEBOOK_UTILS_FOLDER
%store -r MY_NOTEBOOK_VOYAGER_TMP

%store -r MY_UTIL_FUNCTIONS_FILE
%store -r MY_UTIL_YBTSERVER_METRICS_FILE

%store -r BETA_FAST_DATA_EXPORT

---
# Workflow: Migrate data


<div style="width:100%; background-color: #000041"><img src="assets/MVOY100_03_3-Migrate_Data.png" /></div>

---
## Export source data
Export the source data from the source database using `yb-voyager`.  First, review the help for the command, `yb-voyager export data`.


In [ ]:
%%bash

yb-voyager export data --help


> Note:
> 
> The `source-db-schema` argument is not applicable to MySQL.
>
> The `-y` argument says yes to all prompts.
>
> BETA_FAST_DATA_EXPORT=1 is an environment variable.
> 

Run the `export data` command.

In [ ]:
%%bash -s "$MY_MYSQL_DB_NAME"  "$MY_MYSQL_PORT" "$MY_HOST_IPv4_01" "$MY_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}

yb-voyager export data -y  \
  --start-clean true \
  --disable-pb true \
  --send-diagnostics false \
  --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir \
  --source-db-type "MYSQL" \
  --source-db-host ${DB_HOST} \
  --source-db-port  ${DB_PORT} \
  --source-db-name  ${DB_NAME} \
  --source-db-user "ybvoyager" \
  --source-db-password "Yugabyte#1" > /dev/null
  
       

> Problems?
> 
> In case there are issues with the above and you need to kill the process, you can uncomment and run the following:

In [ ]:
%%bash 
# ps -eaf | grep yb-voyager | grep -v grep | awk '{ print $2 }' | xargs kill -9
# ps -aux | grep 'yb-voyager'

Review the status of the export and run the `export data status` command.

In [ ]:
%%bash -s "$MY_MYSQL_DB_NAME"  "$MY_MYSQL_PORT" "$MY_HOST_IPv4_01" "$MY_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}

yb-voyager export data status \
  --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir 


To view more details, open the `export_status.json` file.

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

cat $NOTEBOOK_VOYAGER_TMP/export-dir/data/export_status.json

Review the Voyager log file for the export.

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}
EXPORT_DATA_LOG=yb-voyager-export-data.log

tail -n 200 $NOTEBOOK_VOYAGER_TMP/export-dir/logs/$EXPORT_DATA_LOG
gp open $NOTEBOOK_VOYAGER_TMP/export-dir/logs/$EXPORT_DATA_LOG

---
## Import source data
Import the source data to the target YugabyteDB database using the `yb-voyager import data` command.

In [ ]:
%%bash -s "$MY_DB_NAME"  "$MY_DB_PORT" "$MY_HOST_IPv4_01" "$MY_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}

yb-voyager import data  \
  --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir \
  --target-db-host ${DB_HOST} \
  --target-db-port  ${DB_PORT} \
  --target-db-name  ${DB_NAME} \
  --target-db-user "ybvoyager" \
  --target-db-password "Password" \
  --target-db-schema "sakila" > /dev/null

You can check the status of the import.

In [ ]:
%%bash -s "$MY_MYSQL_DB_NAME"  "$MY_MYSQL_PORT" "$MY_HOST_IPv4_01" "$MY_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}

yb-voyager import data status \
  --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir 


> Did you know?
>
> You can import data from cloud storage such as:
> - AWS S3 
> - GCS buckets
> - Azure blob 
> 
> For more details, review [Import Data File](https://docs.yugabyte.com/preview/yugabyte-voyager/migrate-steps/#import-data-file) in Voyager documentation.
> 


---
## Post import data
After importing the data, rerun the import schema command, but this time, using the arguments, `post-import-data` and `refresh-mviews`.

The `post-import-data` argument creates indexes, foreign keys, and triggers in the target database. 

The `refresh-mviews` argument instructs Voyager to refresh any materialized views on the target database during the post import data phase. 

In [ ]:
%%bash -s "$MY_DB_NAME"  "$MY_DB_PORT" "$MY_HOST_IPv4_01" "$MY_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}
IMPORT_SCHEMA_LOG=yb-voyager-import-schema.log

# delete the log so that we can get a fresh view
rm -rf $NOTEBOOK_VOYAGER_TMP/export-dir/logs/$IMPORT_SCHEMA_LOG

yb-voyager import schema  -y  \
  --post-import-data true \
  --refresh-mviews true \
  --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir  \
  --target-db-host ${DB_HOST}  \
  --target-db-port  ${DB_PORT}  \
  --target-db-name  ${DB_NAME}  \
  --target-db-user "ybvoyager"  \
  --target-db-password "Password"  \
  --target-db-schema "sakila" > /dev/null
  

---
## Verify migration
After completing the migration of the schema and the data,the automated migration is finished. Next is the manual verification of the migration success. 

You need to manually validate the successful migration of database objects, including the post import data objects such as triggers and indexes.

You also need to manually run validation queries on both the source and target database to ensure that the data is correctly migrated. A sample query to validate the databases can include checking the row count of each table.


### Connect to YugabyteDB using the PostgreSQL Driver for Python

In [ ]:
# Connect to db_ybu
# Inspiration from https://medium.com/analytics-vidhya/postgresql-integration-with-jupyter-notebook-deb97579a38d
import psycopg2
import sqlalchemy as alc
from sqlalchemy import create_engine

# env_var.env
db_host=MY_HOST_IPv4_01
db_name=MY_DB_NAME

connection_str='postgresql://yugabyte@'+db_host+':5433/'+db_name
print(connection_str)
engine = create_engine(connection_str)

#### Load SQL magic extension
>IMPORTANT!
>
> To use SQL magic, you must run the following cell that loads the notebook extension.

In [ ]:
%reload_ext sql
# creates connection for sql magic
%sql {connection_str}

### Run queries to validate objects

Verify the creation of indexes.

In [ ]:
%%sql
select tablename, indexname, indexdef from pg_indexes where schemaname = 'sakila' order by tablename, indexname;

Verify the creation of triggers.

In [ ]:
%%sql
select  event_object_table AS table_name, trigger_name from information_schema.triggers where trigger_schema = 'sakila' group by table_name, trigger_name order by table_name, trigger_name;

### Run queries

In [ ]:
%%sql
select * from sakila.actor where UPPER(first_name) = 'SCARLETT';

In [ ]:
%%sql
select count(distinct last_name) from sakila.actor;

In [ ]:
%%sql
select actor.actor_id, actor.first_name, actor.last_name,
       count(actor_id) as film_count
from sakila.actor join sakila.film_actor using (actor_id)
group by actor_id
order by film_count desc
limit 1;

### Cleanup the migration user

In [ ]:
%%sql 

REASSIGN OWNED BY ybvoyager TO yugabyte;
DROP OWNED BY ybvoyager;
DROP USER ybvoyager;

---

## 😊 Next up!
Continue your learning by opening the next notebook, `06_Migration_Workflow_End_Migration.ipynb`. 

Or, to open the notebook from GitPod, run the following:

In [ ]:
%%bash
gp open '06_Migration_Workflow_End_Migration.ipynb'